In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of text: {} character'.format(len(text)))

Length of text: 1115394 character


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
print("{")
for char, _ in zip(char2idx, range(20)):
    print(" {:4s}: {:3d},".format(repr(char), char2idx[char]))
print(" ...\n")

{
 '\n':   0,
 ' ' :   1,
 '!' :   2,
 '$' :   3,
 '&' :   4,
 "'" :   5,
 ',' :   6,
 '-' :   7,
 '.' :   8,
 '3' :   9,
 ':' :  10,
 ';' :  11,
 '?' :  12,
 'A' :  13,
 'B' :  14,
 'C' :  15,
 'D' :  16,
 'E' :  17,
 'F' :  18,
 'G' :  19,
 ...



In [8]:
print("{} ---- characters mapped to int ---- > {}".format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
seq_length = 100
examples_per_epoch = len(text) // seq_length

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [15]:
for input_example, target_example in dataset.take(1):
    print("Input data: ", repr(''.join(idx2char[input_example.numpy()])))
    print("Target data: ", repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [16]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print(" input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print(" output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
 input: 18 ('F')
 output: 47 ('i')
Step    1
 input: 47 ('i')
 output: 56 ('r')
Step    2
 input: 56 ('r')
 output: 57 ('s')
Step    3
 input: 57 ('s')
 output: 58 ('t')
Step    4
 input: 58 ('t')
 output: 1 (' ')


In [17]:
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch // BATCH_SIZE

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [0]:
if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(
        tf.keras.layers.GRU,
        recurrent_activation='sigmoid'
    )

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        rnn(rnn_units, 
            return_sequences=True,
            recurrent_initializer='glorot_uniform', 
            stateful=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [0]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE
)

In [22]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [25]:
sampled_indices

array([10, 49, 28, 10, 21, 56, 57,  8, 61,  6, 21, 25, 62, 15, 39, 41, 42,
       11, 15, 14, 57,  7, 19,  5, 24, 17, 49, 16, 47, 57, 47, 23,  8, 43,
       47, 28, 50, 62, 20,  1, 59, 55, 47, 29, 55, 20, 10, 63, 17, 45, 32,
       38, 11, 38, 29, 52, 44, 17, 23, 52, 52,  6, 38, 36, 39, 23, 28, 48,
       35,  9, 31, 40, 64, 48, 20,  9, 34, 60, 15, 29, 47, 32,  8, 60, 12,
       41,  8, 26,  1, 11, 25, 57, 21,  2, 33,  5, 55, 27, 43, 24])

In [26]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr(''.join(idx2char[sampled_indices])))

Input: 
 'breathed your curse against yourself.\n\nQUEEN MARGARET:\nPoor painted queen, vain flourish of my fortu'

Next Char Predictions: 
 ":kP:Irs.w,IMxCacd;CBs-G'LEkDisiK.eiPlxH uqiQqH:yEgTZ;ZQnfEKnn,ZXaKPjW3SbzjH3VvCQiT.v?c.N ;MsI!U'qOeL"


In [0]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [28]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Scalar loss: ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
Scalar loss:  4.1732845


In [0]:
model.compile(
    optimizer=tf.train.AdamOptimizer(),
    loss=loss
)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [0]:
EPOCHS=30

In [32]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/30
174/174 [==============================] - 28s 160ms/step - loss: 2.6896
Epoch 2/30
174/174 [==============================] - 25s 146ms/step - loss: 1.9318
Epoch 3/30
174/174 [==============================] - 25s 146ms/step - loss: 1.6717
Epoch 4/30
174/174 [==============================] - 25s 146ms/step - loss: 1.5298
Epoch 5/30
174/174 [==============================] - 25s 146ms/step - loss: 1.4452
Epoch 6/30
174/174 [==============================] - 25s 146ms/step - loss: 1.3868
Epoch 7/30
174/174 [==============================] - 26s 147ms/step - loss: 1.3420
Epoch 8/30
174/174 [==============================] - 25s 146ms/step - loss: 1.3045
Epoch 9/30
174/174 [==============================] - 25s 146ms/step - loss: 1.2708
Epoch 10/30
174/174 [==============================] - 25s 146ms/step - loss: 1.2390
Epoch 11/30
174/174 [==============================] - 25s 146ms/step - loss: 1.2069
Epoch 12/30
174/174 [==============================] - 25s 146ms/step - lo

In [33]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
    num_generate = 1000
    
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    
    temperature = 1.0
    
    model.reset_states()
    
    for i in range(num_generate):
        predictions = model(input_eval)
        
        predictions = tf.squeeze(predictions, 0)
        
        predictions = predictions / temperature
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1, 0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        
        text_generated.append(idx2char[predicted_id])
        
    return (start_string + ''.join(text_generated))

In [37]:
print(generate_text(model, start_string=u"ROMEO: "))

W0708 16:17:39.771296 139856563066752 deprecation.py:323] From <ipython-input-36-c43f35a8fc58>:19: multinomial (from tensorflow.python.ops.random_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.random.categorical` instead.


ROMEO: good sir, we are
Will never fall balm and sweeter bastards:
Her one as Your holds me speak.

SICINIUS:
Mercy isld,
And overgate to my bed, o' the gate!

KING HENRY VI:
And shall sake the window'st perform; and Mantua
Noy to report the chamble?

CATESBY:
We meas'd in mine have done!

PRINCE EDWARD:
Nay, mark her knowing? I must have, must die?

LADY GREY:
God for a hire is once. When he sits done
That thou hast wrought;
And by the right and state as 'twere no crown,
To make thee cunning of devoted profes,
thou talk'st
Upon thy desent daughter: one being so remain?

SLY:
I'll give thee mercy then.
Be
Whose honourable house this my dreams;
Which before merciful's, things but a traitor, thou there lies
Till titting joy
That he drews on this afternoon; 'tis there an injury,
That it should seem as scarce had been
Paris simple the music of sworn and aid
And level at my countenance of this sictle time
Where being treasure here of remorse;
Which seem she's dead, and have absent.

LUCENTI

In [0]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = BATCH_SIZE
)

In [0]:
optimizer = tf.train.AdamOptimizer()

In [43]:
EPOCHS = 1

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = model.reset_states()
    
    for (batch_n, (inp, target)) in enumerate(dataset):
        with tf.GradientTape() as tape:
            predictions = model(inp)
            loss = tf.losses.sparse_softmax_cross_entropy(target, predictions)
            
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        if batch_n % 100 == 0:
            template = "Epoch {} Batch {} Loss {:.4f}"
            print(template.format(epoch+1, batch_n, loss))
            
model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.1743
Epoch 1 Batch 100 Loss 2.3788
